# Replicating *Why Do Couples and Singles Save during Retirement?* – Model Setup  
This notebook walks through a step‑by‑step replication of the structural life‑cycle model developed by De Nardi et al. (2025).  

**Goal of this first step**

* Gather every constant that the authors calibrate or estimate (as reported in Table 5 and in the text)  
* Define them in a single `parameters` dictionary we can reuse later  

**Paper reference**
> cite: De Nardi, M., E. French, J. B. Jones & R. McGee (2025): “Why Do Couples and Singles Save during Retirement? Household Heterogeneity and Its Aggregate Implications”, *Journal of Political Economy*, 133(3). 
 


## Core preference and policy parameters  

| Symbol | Meaning | Value |
|--------|---------|-------|
| $h$ | Consumption‑equivalence scale (couples vs singles) | 1.528 |
| $\rho$ | Coefficient of relative risk aversion ($n$ in the paper) | 3.701 |
| $\phi_0$ | Bequest intensity when **no** survivor | 6 826 (×10³) |
| $\kappa_0$ | Bequest curvature when **no** survivor | 3 517 (×10³) |
| $\phi_1$ | Bequest intensity when a **surviving spouse** exists | 4 447 |
| $\kappa_1$ | Bequest curvature when a **surviving spouse** exists | 211.2 (×10³) |
| $c_{\min}^{S}$ | Annual consumption floor – singles | \$4 101 |
| $c_{\min}^{C}$ | Couples’ floor | 1.5 × $c_{\min}^{S}$ |
| $\beta$ | Annual discount factor | 0.97 |
| $\beta_{2y}$ | Two‑year discount factor (model period) | $0.97^{2}=0.9409$ |

> **Note:** The paper states only that the real pretax return $r$ is “constant” but does not pin down its value.  
We leave it as a to‑do placeholder and will set it once we locate it in the appendices.  


In [4]:
import math
import numpy as np
import pandas as pd

In [5]:

# --- Core parameters (from Table 5) ---
parameters = {
    # Preference
    "beta_annual": 0.97,                     # annual discount factor
    "beta": 0.97 ** 2,                       # two‑year period (AHEAD wave spacing)
    "h": 1.528,                              # equivalence scale
    "rho": 3.701,                            # CRRA (n in paper)
    # Bequest function v_j(b) = phi_j * (b + kappa_j)^(1 - rho) / (1 - rho)
    "phi0": 6_826_000,                       # intensity – terminal bequests
    "kappa0": 3_517_000,                     # curvature – terminal bequests
    "phi1": 4_447_000,                       # intensity – side bequests
    "kappa1": 211_200,                       # curvature – side bequests
    # Consumption floor
    "cmin_single_annual": 4_101,
    "cmin_couple_annual": 1.5 * 4_101,
    # Placeholder for economic environment
    "r": None,                             
}

In [7]:
# fill in the missing pretax return (2 % real annual ⇒ 2‑year gross factor ≈ 1.0404)
parameters["r_annual"] = 0.02
parameters["R"]        = (1 + parameters["r_annual"]) ** 2      # two‑year gross factor#

In [9]:
print(parameters)

{'beta_annual': 0.97, 'beta': 0.9409, 'h': 1.528, 'rho': 3.701, 'phi0': 6826000, 'kappa0': 3517000, 'phi1': 4447000, 'kappa1': 211200, 'cmin_single_annual': 4101, 'cmin_couple_annual': 6151.5, 'r': None, 'r_annual': 0.02, 'R': 1.0404}


### Utility & bequest functions
The paper uses CRRA utility and the “ luxury‑good ” bequest specification  
$
u^S(c)=\\frac{c^{1-\\rho}}{1-\\rho},\\qquad  
u^C(c)=2\\,\\frac{\\bigl(c/h\\bigr)^{1-\\rho}}{1-\\rho},\\qquad  
v_j(b)=\\phi_j\\,\\frac{(b+\\kappa_j)^{1-\\rho}}{1-\\rho},\\;j\\in\\{0,1\\}
$

In [10]:
def u_single(c):
    c = np.maximum(c, 1e-10)
    return c**(1 - rho) / (1 - rho)

def u_couple(c):
    h = parameters["h"]
    c = np.maximum(c, 1e-10)
    return 2 * ((c / h) ** (1 - rho)) / (1 - rho)

def v0(b):
    phi = parameters["phi0"]
    k   = parameters["kappa0"]
    return phi * ((b + k) ** (1 - rho)) / (1 - rho)

def v1(b):
    phi = parameters["phi1"]
    k   = parameters["kappa1"]
    return phi * ((b + k) ** (1 - rho)) / (1 - rho)

# quick sanity check
print("u_S(50k) =", u_single(50_000))
print("v0(0)    =", v0(0))

NameError: name 'rho' is not defined